# Лабораторная работа №3:

In [8]:
import numpy as np
from typing import Tuple, Mapping
import random
from numba import njit

In [17]:
@njit
def Matias(x: np.array, y:np.array) -> np.float128:
  return 0.26*(x*x + y*y) - 0.48*x*y
@njit
def Matias_dfdx(x: np.array, y:np.array) -> np.float128:
  return 0.52 * x - 0.48 * y
@njit
def Matias_dfdy(x: np.array, y:np.array) -> np.float128:
  return 0.52 * y - 0.48 * x

In [18]:
@njit
def my_GD(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float, lr: float = 0.01,
          T:int = 1000) -> Tuple [np.ndarray, np.float32]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  lr - скорость обучения (default: 0.01)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0

  for i in range(T):
    xt = xt - lr * dx(xt, yt)
    yt = yt - lr * dy(xt, yt)
  return x0, y0, xt, yt, f(xt, yt)

In [19]:
x0, y0, x, y, z = my_GD(Matias, Matias_dfdx, Matias_dfdy, 10, -10, lr = 0.1)
print("Градиентный спуск\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", x, ", yt =", y,
      "\nЗначение функции в точке глобального минимума: f(xt, yt) =", z)

Градиентный спуск
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = -0.004229136952113594 , yt = -0.004220461571001646 
Значение функции в точке глобального минимума: f(xt, yt) = 7.139759675776723e-07


In [20]:
@njit
def my_GD_Momentum(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float, 
        beta: float = 0.9, T:int = 1000) -> Tuple [int, int, float, float, float]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  beta - коэффициент обучения (default: 0.9)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0

  for i in range (T):
    xt = xt * beta - (1 - beta) * dx(xt, yt)
    yt = yt * beta - (1 - beta) * dy(xt, yt)
  return x0, y0, xt, yt, f(xt, yt)

In [21]:
x0, y0, p_x, p_y, p_z = my_GD_Momentum(Matias, Matias_dfdx, Matias_dfdy, x0=10, y0=-10)
print("Алгоритм GD Momentum\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", p_x, ", yt =", p_y,
      "\nГлобальный минимум: f(xt, yt) =", p_z)

Алгоритм GD Momentum
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = -5.990194208112103e-50 , yt = -5.66182284098101e-50 
Глобальный минимум: f(xt, yt) = 1.3846519518114525e-100


In [22]:
@njit
def my_GD_ADAM(f: Mapping, dx:Mapping, dy:Mapping, x0: float, y0: float,
        lr: float = 0.1, beta1: float = 0.9, beta2: float = 0.99,
        eps: float = 1e-8, T:int = 1000) -> Tuple [int, int, float, float, float]:
  """
  Функция принимает:
  f - функционал для оптимизации
  dx, dy - градиент оптимизируемого функционала
  x0, y0 - стартовая точка алгоритма
  lr - коэффициент скорости обучения (default: 0.1)
  beta1 - параметр (default: 0.9)
  beta2 - параметр (default: 0.99)
  eps - параметр (default: 1e-8)
  T - количество итераций (default: 1000)

  Функция возвращает:
  Кортеж, включающий в себя: координаты сгенерированной случайно начальной точки,
  координаты полученной точки глобального минимума, значение функции в найденной
  точке глобального минимума
  """
  xt = x0
  yt = y0
  vtx = 0
  vty = 0
  Gtx = 0
  Gty = 0

  for i in range (T):
    grad_x = dx(xt, yt)
    grad_y = dy(xt, yt)
    vtx = beta1 * vtx + (1 - beta1) * grad_x
    vty = beta1 * vty + (1 - beta1) * grad_y
    Gtx = beta2 * Gtx + (1 - beta2) * (grad_x) ** 2
    Gty = beta2 * Gty + (1 - beta2) * (grad_y) ** 2
    xt = xt - lr * vtx / (Gtx + eps) ** (1 / 2)
    yt = yt - lr * vty / (Gty + eps) ** (1 / 2)
  return x0, y0, xt, yt, f(xt, yt)

In [23]:
x0, y0, p_x, p_y, p_z = my_GD_ADAM(Matias, Matias_dfdx, Matias_dfdy, x0=10, y0=-10)
print("Алгоритм GD ADAM\nНачальная точка: x0 =", x0, ", y0 =", y0,
      "\nТочка глобального минимума: xt =", p_x, ", yt =", p_y,
      "\nГлобальный минимум: f(xt, yt) =", p_z)

Алгоритм GD ADAM
Начальная точка: x0 = 10 , y0 = -10 
Точка глобального минимума: xt = -8.08304859034943e-22 , yt = 8.08304859034943e-22 
Глобальный минимум: f(xt, yt) = 6.533567451394991e-43
